In [ ]:
! pip install wandb opencv-python-headless==4.1.2.30 albumentations torch-summary timm==0.5.4 einops joblib icecream  -qq -U

     |████████████████████████████████| 1.7 MB 12.2 MB/s 
     |████████████████████████████████| 21.8 MB 1.6 MB/s 
     |████████████████████████████████| 102 kB 39.0 MB/s 
     |████████████████████████████████| 431 kB 43.8 MB/s 
     |████████████████████████████████| 97 kB 5.5 MB/s 
     |████████████████████████████████| 142 kB 47.5 MB/s 
     |████████████████████████████████| 180 kB 45.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [ ]:
import warnings
from glob import glob
import pathlib
from pathlib import Path
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pprint import pprint
import urllib.request
import csv
import numpy as np
from einops import rearrange, reduce, repeat
from torch.cuda import amp
from tqdm import tqdm
import wandb
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os
from icecream import ic
from sklearn.model_selection import train_test_split
import gc
import cv2
import copy
import time
import random
from PIL import Image

# For data manipulation
import numpy as np
import pandas as pd
from einops import rearrange, repeat
from scipy import stats

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

import timm

import json

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


# ENV

In [ ]:
# Option for Mixed Precision
# FP16 = True
FP16 = False


CONFIG = dict(
    nickname = 'Ensemble 9',
    seed=42,
    backbone=None,
    embedder=None,
    train_batch_size=8,
    valid_batch_size=16,
    img_size=384,
    num_epochs=5,
    early_stopping=False,
    early_stopping_step=10,
    learning_rate=1e-4,
    scheduler='CosineAnnealingLR',
    min_lr=1e-6,
    T_max=100,
    num_classes=25,
    weight_decay=1e-6,
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    competition='lg',
    _wandb_kernel='deb'
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed(CONFIG['seed'])


In [ ]:
import wandb


run = wandb.init(project="lg",
                 entity="jiwon7258",
                 config=CONFIG,
                 job_type='inf',
                #  id='1l0lpdqx',
                #  resume='must'
)


dataset = wandb.run.use_artifact(
    'jiwon7258/lg/lg:v1', type='dataset')

run.name = CONFIG['nickname']

# Download the artifact's contents
dataset_dir = dataset.download()
dataset_dir = Path(dataset_dir)


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Downloading large artifact lg:v1, 9188.81MB. 121115 files... Done. 0:0:0


# Augmentation

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05,
                           rotate_limit=15, p=0.5),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15,
                   b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Normalize(),
        ToTensorV2(),
        ], p=1.),
}
data_transforms_valid = {
    224: A.Compose([
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()], p=1.),


    384: A.Compose([
        A.Resize(384, 384),
        A.Normalize(),
        ToTensorV2()], p=1.),


    # 512: A.Compose([
    #     A.Resize(512, 512),
    #     A.Normalize(),
    #     ToTensorV2()], p=1.)
}


# Dataset Dataloader

In [ ]:
# dataset_dir = Path('./')
TEST_PATH =  dataset_dir / 'test'

In [ ]:
test_csv = sorted(glob(str(TEST_PATH / '*/*.csv')))
test_jpg = sorted(glob(str(TEST_PATH / '*/*.jpg')))

In [ ]:
# # TTA
# class TestDataset(Dataset):
#     def __init__ (self, N, test_imgs, transforms = None):
#         self.N = N
#         self.test_imgs = test_imgs
#         self.transforms = transforms


#     def __len__ (self):
#         return len(self.test_imgs)

#     def __getitem__(self, index):
#         img_path = self.test_imgs[index]
#         img_code = (Path(img_path)).parent.stem
#         img = cv2.imread(img_path)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
#         imgs = [self.transforms(image=img)['image'] for _ in range(self.N)]
#         imgs = np.concatenate(imgs, axis = 0)
#         imgs = rearrange(imgs, '(new1 bs) h w -> new1 bs h w', new1=self.N)

#         return imgs, img_code

# testDataset = TestDataset(8, test_jpg, transforms = data_transforms['train'])
# testDataloader = DataLoader(
#     testDataset, batch_size=CONFIG['valid_batch_size'], shuffle=False)


In [ ]:
class TestDataset(Dataset):
    def __init__(self, test_imgs, transforms=None):
        self.test_imgs = test_imgs
        self.transforms = transforms

    def __len__(self):
        return len(self.test_imgs)

    def __getitem__(self, index):
        img_path = self.test_imgs[index]
        img_code = (Path(img_path)).parent.stem
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        img_list = [transform(image=img)['image'] for transform in self.transforms.values()]
        

        return img_list, img_code

# 384
testDataset = TestDataset(test_jpg, transforms=data_transforms_valid)
testDataloader = DataLoader(
    testDataset, batch_size=CONFIG['valid_batch_size'], shuffle=False)


In [ ]:
# for imgs, img_code in testDataset :
#     fig, axes = plt.subplots(1,8, figsize = (40,20))
#     for i in range(8):
#         img = imgs[i, :, :, :]
#         axes[i].imshow(img.reshape(384,384,3))
#     break

In [ ]:
for img_list, img_code in testDataloader:
    for i in range(len(img_list)):
        print(img_list[i].shape)
    print(type(img_code))
    break


torch.Size([16, 3, 224, 224])
torch.Size([16, 3, 384, 384])
torch.Size([16, 3, 512, 512])
<class 'tuple'>


# Key Value Dict

In [ ]:
wandb.restore('class_dict', run_path='jiwon7258/lg/1lkvc6n0', root='./')
class_dict = joblib.load('class_dict')

# MODELS

## EfficientNet V2 M

In [ ]:
class Effnet(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = 1280
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        output = self.fc(features)
        return output


effnet = Effnet('tf_efficientnetv2_m',None, pretrained=False)
effent = effnet.to(CONFIG['device'])
;

''

In [ ]:
wandb.restore('tf_efficientnetv2_m finecutmixlast.bin',
              'jiwon7258/lg/lwda8bn3', root='./')
effnet.load_state_dict(torch.load('tf_efficientnetv2_m finecutmixlast.bin',
                      map_location=CONFIG['device']))


<All keys matched successfully>

## Swin B 

In [25]:
class Swin(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Swin, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.n_features = self.backbone.head.in_features
        self.backbone.reset_classifier(0)
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        output = self.fc(features)
        return output


swin = Swin('swin_base_patch4_window12_384',
            None, pretrained=False)
swin.to(CONFIG['device'])
;

''

In [27]:
wandb.restore('epoch125_Loss0.0002.bin', 'jiwon7258/lg/1ycjalgj', root='./')
swin.load_state_dict(torch.load('epoch125_Loss0.0002.bin',
                      map_location=CONFIG['device']))


<All keys matched successfully>

## DeiT

In [28]:
class Deit(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Deit, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = 768
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        if isinstance(features, tuple):
          features = features[0]
        output = self.fc(features)
        return output


deit = Deit('deit_base_distilled_patch16_384',
            None, pretrained=False)
deit.to(CONFIG['device'])
;

''

In [29]:
wandb.restore('deit finecutmixlast.bin', 'jiwon7258/lg/5ikpl9uv', root='./')
deit.load_state_dict(torch.load(
    'deit finecutmixlast.bin', map_location=CONFIG['device']))


<All keys matched successfully>

## EfficientNet B4 NS 512

In [30]:
class Noisy(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Noisy, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = 1792
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        # if isinstance(features, tuple):
        #   features = features[0]
        output = self.fc(features)
        return output


b4ns512 = Noisy('tf_efficientnet_b4_ns', None, pretrained=False)
b4ns512.to(CONFIG['device'])
;

''

In [31]:
wandb.restore('epoch71_Loss0.0003.bin', 'jiwon7258/lg/1v3ffaqb', root='./')
b4ns512.load_state_dict(torch.load(
    'epoch71_Loss0.0003.bin', map_location=CONFIG['device']))


<All keys matched successfully>

## CoAtNet Mini 224

In [32]:
class Coat(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Coat, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = self.backbone.num_features
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        # if isinstance(features, tuple):
        #   features = features[0]
        output = self.fc(features)
        return output


coatmini224 = Coat('coat_mini', None, pretrained=False)
coatmini224.to(CONFIG['device'])
;

''

In [33]:
wandb.restore('CoAtNet mini finecutmixlast.bin',
              'jiwon7258/lg/zwfbinnt', root='./')
coatmini224.load_state_dict(torch.load(
    'CoAtNet mini finecutmixlast.bin', map_location=CONFIG['device']))


<All keys matched successfully>

## BEiT 224 in 22k

In [ ]:
class Beit(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Beit, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = self.backbone.num_features
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        # if isinstance(features, tuple):
        #   features = features[0]
        output = self.fc(features)
        return output


beit224 = Beit('beit_base_patch16_224_in22k',
            None,
             pretrained=False)
beit224.to(CONFIG['device']);

In [35]:
wandb.restore('BEiT 224 in22k finecutmixlast.bin',
              'jiwon7258/lg/288jowyh', root='./')
beit224.load_state_dict(torch.load(
    'BEiT 224 in22k finecutmixlast.bin', map_location=CONFIG['device']))


<All keys matched successfully>

## CaiT S24 224

In [ ]:
class Cait(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Cait, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = self.backbone.num_features
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        # if isinstance(features, tuple):
        #   features = features[0]
        output = self.fc(features)
        return output


cait224 = Cait('cait_s24_224',
            None,
            pretrained=False)
cait224.to(CONFIG['device']);

In [37]:
wandb.restore('last.bin', 'jiwon7258/lg/5xzq9850', root='./')
cait224.load_state_dict(torch.load(
    'last.bin', map_location=CONFIG['device']))


<All keys matched successfully>

## Swin S 224

In [38]:
class SwinS(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(SwinS, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = self.backbone.num_features
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        # if isinstance(features, tuple):
        #   features = features[0]
        output = self.fc(features)
        return output


swinS224 = SwinS('swin_small_patch4_window7_224', None, pretrained=False)
swinS224.to(CONFIG['device']);


In [40]:
MODEL_NAME = 'swin_small_224 finecutmixlast.bin'
RUN_PATH = 'jiwon7258/lg/3199pcx4'
wandb.restore(MODEL_NAME, RUN_PATH, root='./')
swinS224.load_state_dict(torch.load(
    MODEL_NAME, map_location=CONFIG['device']))


<All keys matched successfully>

## ConvNext Small 224

In [41]:
class ConvnextS(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(ConvnextS, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = self.backbone.num_features
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        # if isinstance(features, tuple):
        #   features = features[0]
        output = self.fc(features)
        return output


convnextS224 = ConvnextS('convnext_small', None, pretrained=False)
convnextS224.to(CONFIG['device'])


ConvnextS(
  (backbone): ConvNeXt(
    (stem): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (stages): Sequential(
      (0): ConvNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvNeXtBlock(
            (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=96, out_features=384, bias=True)
              (act): GELU()
              (drop1): Dropout(p=0.0, inplace=False)
              (fc2): Linear(in_features=384, out_features=96, bias=True)
              (drop2): Dropout(p=0.0, inplace=False)
            )
            (drop_path): Identity()
          )
          (1): ConvNeXtBlock(
            (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups

In [42]:
MODEL_NAME = 'convnext_small finecutmixlast.bin'
RUN_PATH = 'jiwon7258/lg/2ow4king'
wandb.restore(MODEL_NAME, RUN_PATH, root='./')
convnextS224.load_state_dict(torch.load(
    MODEL_NAME, map_location=CONFIG['device']))


<All keys matched successfully>

In [43]:
models = {
    224 : [],
    384 : [],
    # 512 : [],
}
models[224].append(coatmini224)
models[224].append(convnextS224)
models[224].append(swinS224)
models[224].append(cait224)
models[224].append(beit224)
models[384].append(effnet)
models[384].append(swin)
models[384].append(deit)
# models[512].append(b4ns512)

In [44]:
# # TTA
# img_code_list = []
# outputs = []
# with torch.no_grad():
#     model.eval()

#     bar = tqdm(enumerate(testDataloader), total=len(testDataloader))

#     for step, (imgs, img_codes) in bar:
#         # imgs : (bs, N, C, H, W), torch.Tensor
#         # img_codes (bs, N), list(str)
#         imgs = imgs.to(CONFIG['device'])
#         imgs = rearrange(imgs, 'bs N C H W -> (bs N) C H W')
#         logits = model(imgs)
#         # logits : ( (bs * N), num_classes)
#         logits = logits.detach().cpu()
#         logits = rearrange(
#             logits, '(bs N) num_classes -> bs N num_classes', bs=CONFIG['valid_batch_size'])
#         output = np.argmax(torch.softmax(logits, dim=-1), axis=-1)
#         output = np.array(output)
#         # (bs, N)
#         output = stats.mode(output, axis=1)[0].reshape(-1)
#         # (bs)
#         output = [class_dict[i] for i in output]

#         print(img_codes)
#         print(output)

#         img_code_list.extend(img_codes)
#         outputs.extend(output)


# Inference

In [45]:
# no TTA
with torch.no_grad():

    for model_list in models.values():
        for model in model_list :
            model.eval()

    img_code_list = []
    outputs = []
        
    bar = tqdm(enumerate(testDataloader), total=len(testDataloader))

    for step, (img_list, img_code) in bar:
        # img : list of (bs, C, H, W)
        # img_code : (bs)
        
        probs_list = []
        # list of (bs, num_classes)

        for i, model_list in enumerate(models.values()):
            img= img_list[i]
            img = img.to(device)
            batch_size = img.shape[0]
        
            for model in model_list:
                logits = model(img)
                logits = logits.cpu()
                # (bs, num_classes)
                # 모델들마다 logit들의 값의 크기가 다를 수 있으므로, softmax를 이용해 정규화하는 작업이 필요하다
                prob = torch.softmax(logits, dim = -1)
                # (bs, num_classes)
                probs_list.append(prob)
            
        probs_list = torch.stack(probs_list)
        # (num_model, bs, num_clases)
        probs = torch.sum(probs_list, dim = 0)
        # (bs, num_classes)

        output = np.argmax(torch.softmax(probs, dim=-1), axis=-1) # torch.Tensor, (bs,)
        output = np.array(output)
        output = [class_dict[i] for i in output]

        img_code_list.extend(img_code)
        outputs.extend(output)

        assert len(img_code_list) == len(outputs)




100%|██████████| 3245/3245 [2:01:27<00:00,  2.25s/it]


In [48]:
result = pd.DataFrame()
result['image'] = img_code_list
result['label'] = outputs
SUBMIT_FILE_NAME = 'finecutmix_effnetv2m_swin_deit_coatmini224_beit224in22k_cait224_swinS224_convnextS224.csv'
result.to_csv(SUBMIT_FILE_NAME,index= False, index_label=False)

In [49]:
wandb.save(SUBMIT_FILE_NAME)


[]

In [50]:
wandb.finish()